In [1]:
import requests
from bs4 import BeautifulSoup as bs
# import pandas as pd
import json
import tqdm
from os import mkdir
from os.path import join
from shutil import rmtree

In [84]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}
META_PREFICS = "https://www.metacritic.com"
USER_POSTFICS = "/user-reviews"
CRITIC_POSTFICS = "/critic-reviews"

save_path ="/Users/fedor/Desktop/meta_project/data/metacritic/critic_review/"
strange_game_list = []

In [105]:

def parse_critic_page(URL:str,
                       local_save_path:str,
                       json_save:bool = False):

    r = requests.get(URL, headers=headers)
    soup = bs(r.text, "html.parser")

    if soup.find(class_="body product_reviews") is not None:
        base = soup.find(class_="body product_reviews")
    else:
        strange_game_list.append(URL)
        return

    titels = base.find_all(class_="review_content")
    rewiev_count = 0
    for t in titels:

        name = t.find(class_="source").text
        ref =  t.find(class_="external")

        if ref is not None:
            ref = ref['href']
        else:
            ref = None

        score = t.find(class_="review_grade").div.text
        date = t.find(class_="date")

        if date is not None:
            date = date.text
        else:
            date = None

        text = t.find(class_="review_body").text

        if json_save:
            with open( local_save_path +f"/review_{rewiev_count}.json", 'w') as f:
                json.dump({ "name": name, "ref": ref, "score": score,
                            "date":date, "text":text}, f)
        else:
            return { "name": name, "ref": ref, "score": score, "date":date, "text":text}
        rewiev_count +=1


In [86]:
games_name_changed = []
with open('/Users/fedor/Desktop/meta_project/data/metacritic/url/critic_refs/final_critic_refs.json') as f:
    critic_ref = json.load(f)

In [106]:
strange_game_list = []

In [132]:
actual_start_pos = 5005+1129+4095+746

In [134]:
loc_path = save_path
for game_name in tqdm.tqdm(list(critic_ref.keys())[actual_start_pos : ]):
    safe_game_name = game_name.replace("/","_")

    if safe_game_name != game_name:
        games_name_changed.append(game_name)

    try:
        mkdir( loc_path + safe_game_name )
    except FileExistsError:
        rmtree(join(loc_path + safe_game_name))
        mkdir(loc_path + safe_game_name)


    for platform in critic_ref[game_name]:
        if platform == "iPhone/iPad":
            safe_platform = "iPhone_iPad"
        else:
            safe_platform = platform

        mkdir(loc_path+safe_game_name+"/"+safe_platform)

        game_link = critic_ref[game_name][platform]
        parse_critic_page(game_link
                          ,loc_path + safe_game_name + "/" + safe_platform
                          , True)



100%|██████████| 1952/1952 [59:30<00:00,  1.83s/it] 


In [155]:
strange_game_list

['https://www.metacritic.com/game/3ds/3d-fantasy-zone-ii-w/critic-reviews',
 'https://www.metacritic.com/game/3ds/3d-fantasy-zone-ii-w/critic-reviews',
 'https://www.metacritic.com//game/switch/the-longing/critic-reviews',
 'https://www.metacritic.com//game/pc/xianjian-qixia-zhuan-7/critic-reviews',
 'https://www.metacritic.com/game/playstation-2/pinball-hall-of-fame-the-gottlieb-collection/critic-reviews',
 'https://www.metacritic.com/game/pc/el-shaddai-ascension-of-the-metatron/critic-reviews',
 'https://www.metacritic.com/game/xbox-one/voodoo-vince-remastered/critic-reviews',
 'https://www.metacritic.com//game/playstation-2/pinball-hall-of-fame-the-gottlieb-collection/critic-reviews',
 'https://www.metacritic.com/game/gamecube/nfl-quarterback-club-2002/critic-reviews',
 'https://www.metacritic.com//game/playstation-2/nfl-quarterback-club-2002/critic-reviews',
 'https://www.metacritic.com/game/playstation/star-wars-demolition/critic-reviews',
 'https://www.metacritic.com//game/playst

In [149]:
from pathlib import Path
from os.path import getsize
game_count = 0
r_count = 0
bytes_size = 0

# assign directory
directory = "/Users/fedor/Desktop/meta_project/data/metacritic/critic_review"

# iterate over files in
# that directory
names = Path(directory).glob('*')
for n in names:
    rev = Path(n).glob('*')
    game_count += 1
    for r in rev:
        r = Path(r).glob('*')
        for _ in r:
            r_count += 1
            bytes_size += getsize(_)

game_count, r_count, bytes_size


(12899, 470656, 207224215)

In [154]:
bytes_size / (1024**3)

0.19299258943647146